# Title

In [1]:
import tensorflow as tf

In [2]:
from tensorflow.core.framework import graph_pb2

### Load OpenPose graph (with VGG)

In [3]:
tf.reset_default_graph()

In [4]:
openpose_vgg_graph = tf.Graph()

In [5]:
graph_openpose_vgg_def = graph_pb2.GraphDef()
with open("models/openpose_vgg_opt.pb", "rb") as f:
    graph_openpose_vgg_def.ParseFromString(f.read())

In [6]:
with openpose_vgg_graph.as_default():
    tf.import_graph_def(graph_openpose_vgg_def, name="")

In [7]:
with openpose_vgg_graph.as_default():
    tf.import_graph_def(graph_openpose_vgg_def, name="")
    
    input_vgg = openpose_vgg_graph.get_tensor_by_name("inputs:0")
    
    concat_vgg_by_stage = []
    for stage_idx in range(2, 7):
        concat_vgg_by_stage.append(openpose_vgg_graph.get_tensor_by_name('concat_stage{}:0'.format(stage_idx)))

### Load MobileNet OpenPose graph

In [8]:
tf.reset_default_graph()

In [9]:
openpose_mobile_graph = tf.Graph()

In [10]:
graph_openpose_mobile_def = graph_pb2.GraphDef()
with open("models/openpose_mobile_opt.pb", "rb") as f:
    graph_openpose_mobile_def.ParseFromString(f.read())

In [11]:
with openpose_mobile_graph.as_default():
    tf.import_graph_def(graph_openpose_mobile_def, name="")
    
    input_mobile = openpose_mobile_graph.get_tensor_by_name("image:0")
    
    concat_mobile_by_stage = []
    for stage_idx in range(3, 7):
        concat_mobile_by_stage.append(openpose_mobile_graph.get_tensor_by_name('Openpose/MConv_Stage6_concat:0'.format(stage_idx)))
    concat_mobile_by_stage.append(openpose_mobile_graph.get_tensor_by_name('Openpose/concat_stage7:0'))

### All tensors combined

In [12]:
graphs = {
    "vgg": openpose_vgg_graph,
    "mobile": openpose_mobile_graph
}

In [13]:
input_tensors = {
    "vgg": input_vgg,
    "mobile": input_mobile
}

output_tensors = {
    "vgg": concat_vgg_by_stage,
    "mobile": concat_mobile_by_stage
}

In [14]:
import rospy
from sensor_msgs.msg import Image
import cv2, cv_bridge

In [15]:
import numpy as np
import tensorflow as tf
import time

In [16]:
rospy.init_node('openpose')

In [17]:
bridge = cv_bridge.CvBridge()

In [18]:
from estimator import TfPoseEstimator
from helper import Humans

In [19]:
graph = "mobile"
stage_idx = 7
w, h = 320, 240

openpose = TfPoseEstimator(graphs[graph],
                           input_tensors[graph],
                           output_tensors[graph][stage_idx-3],
                           stage=stage_idx,
                           target_size=(w, h)
                          )

In [20]:
def callback(frame_ros):
    frame_cv = bridge.imgmsg_to_cv2(frame_ros, "bgr8")
    
    t = time.time()
    humans = openpose.inference(frame_cv)
    print "Inference time: {}".format(time.time() - t)
    
    humans = Humans(humans, frame_cv)
    
    frame_humans_cv = humans.draw()
    frame_humans_ros = bridge.cv2_to_imgmsg(frame_humans_cv, "bgr8")
    pub.publish(frame_humans_ros)

In [21]:
pub = rospy.Publisher('frame_human', Image, queue_size=1000)

In [22]:
sub = rospy.Subscriber('frame', Image, callback)
rospy.spin()

Inference time: 0.247915029526
Inference time: 0.191650152206
Inference time: 0.183536052704
Inference time: 0.169943094254
Inference time: 0.152652025223
Inference time: 0.153189897537
Inference time: 0.161485910416
Inference time: 0.157696008682
Inference time: 0.162021875381
Inference time: 0.154669046402
Inference time: 0.155611991882
Inference time: 0.150885820389
Inference time: 0.150818109512
Inference time: 0.171268939972
Inference time: 0.213441133499
Inference time: 0.223589897156
Inference time: 0.190538883209
Inference time: 0.18668794632
Inference time: 0.173866033554
Inference time: 0.223807096481
Inference time: 0.229041099548
Inference time: 0.187689065933
Inference time: 0.185695171356
Inference time: 0.154578924179
Inference time: 0.223881959915
Inference time: 0.167725086212
Inference time: 0.16031908989
Inference time: 0.158239126205
Inference time: 0.163302898407
Inference time: 0.157354831696
Inference time: 0.156759023666
Inference time: 0.157488822937
Inference 

Inference time: 0.151523113251
Inference time: 0.150918960571
Inference time: 0.150814056396
Inference time: 0.149730920792
Inference time: 0.151564836502
Inference time: 0.150436878204
Inference time: 0.14990901947
Inference time: 0.152638912201
Inference time: 0.150032043457
Inference time: 0.1486120224
Inference time: 0.151687860489
Inference time: 0.149258852005
Inference time: 0.150480985641
Inference time: 0.151512861252
Inference time: 0.151472091675
Inference time: 0.147285938263
Inference time: 0.148370027542
Inference time: 0.149600028992
Inference time: 0.151118993759
Inference time: 0.152282953262
Inference time: 0.150679111481
Inference time: 0.150832176208
Inference time: 0.150348901749
Inference time: 0.148320913315
Inference time: 0.151663064957
Inference time: 0.150646924973
Inference time: 0.154145956039
Inference time: 0.151433944702
Inference time: 0.149791002274
Inference time: 0.147417068481
Inference time: 0.150798082352
Inference time: 0.15079498291
Inference ti

Inference time: 0.145713090897
Inference time: 0.14691901207
Inference time: 0.147253990173
Inference time: 0.147246837616
Inference time: 0.148103952408
Inference time: 0.146551847458
Inference time: 0.148051023483
Inference time: 0.146700143814
Inference time: 0.148318052292
Inference time: 0.144583940506
Inference time: 0.147999048233
Inference time: 0.145104885101
Inference time: 0.149561882019
Inference time: 0.147927999496
Inference time: 0.148191928864
Inference time: 0.145205974579
Inference time: 0.147853851318
Inference time: 0.145969152451
Inference time: 0.148526191711
Inference time: 0.149847984314
Inference time: 0.149069786072
Inference time: 0.146311044693
Inference time: 0.148458003998
Inference time: 0.143531084061
Inference time: 0.149478912354
Inference time: 0.14574098587
Inference time: 0.148910045624
Inference time: 0.144670009613
Inference time: 0.145867824554
Inference time: 0.147722005844
Inference time: 0.148763179779
Inference time: 0.147612094879
Inference 

Inference time: 0.144169092178
Inference time: 0.146077871323
Inference time: 0.150345802307
Inference time: 0.145955085754
Inference time: 0.146443128586
Inference time: 0.147702932358
Inference time: 0.145385026932
Inference time: 0.147353887558
Inference time: 0.14833688736
Inference time: 0.145792007446
Inference time: 0.146791934967
Inference time: 0.149072170258
Inference time: 0.143928050995
Inference time: 0.147972106934
Inference time: 0.146867990494
Inference time: 0.143989801407
Inference time: 0.147518873215
Inference time: 0.147664070129
Inference time: 0.145375967026
Inference time: 0.148105859756
Inference time: 0.147198915482
Inference time: 0.14369893074
Inference time: 0.14969587326
Inference time: 0.145788192749
Inference time: 0.147702932358
Inference time: 0.147140026093
Inference time: 0.145309209824
Inference time: 0.149477958679
Inference time: 0.14727306366
Inference time: 0.142997980118
Inference time: 0.148262023926
Inference time: 0.144171953201
Inference ti

Inference time: 0.150829792023
Inference time: 0.144654035568
Inference time: 0.147650957108
Inference time: 0.146774053574
Inference time: 0.146750926971
Inference time: 0.148910999298
Inference time: 0.145020961761
Inference time: 0.147733926773
Inference time: 0.149770975113
Inference time: 0.147820949554
Inference time: 0.146730899811
Inference time: 0.144958972931
Inference time: 0.148741960526
Inference time: 0.146002054214
Inference time: 0.147051095963
Inference time: 0.14866900444
Inference time: 0.144757986069
Inference time: 0.148650169373
Inference time: 0.14245390892
Inference time: 0.147888183594
Inference time: 0.142756938934
Inference time: 0.149196863174
Inference time: 0.14405298233
Inference time: 0.147756814957
Inference time: 0.148553133011
Inference time: 0.144155979156
Inference time: 0.147732019424
Inference time: 0.146987915039
Inference time: 0.143519163132
Inference time: 0.148595809937
Inference time: 0.144887924194
Inference time: 0.148413896561
Inference t

...

Inference time: 0.148597955704
Inference time: 0.144917011261
Inference time: 0.149361133575
Inference time: 0.151395082474
Inference time: 0.1442258358
Inference time: 0.14755821228
Inference time: 0.146400928497
Inference time: 0.147938966751
Inference time: 0.148298978806
Inference time: 0.145094156265
Inference time: 0.149709224701
Inference time: 0.144973039627
Inference time: 0.146824121475
Inference time: 0.147225141525
Inference time: 0.146141052246
Inference time: 0.148375034332
Inference time: 0.149224996567
Inference time: 0.14693903923
Inference time: 0.150960922241
Inference time: 0.145617961884
Inference time: 0.147366046906
Inference time: 0.147228002548
Inference time: 0.144315004349
Inference time: 0.150974035263
Inference time: 0.145748138428
Inference time: 0.147567987442
Inference time: 0.148186922073
Inference time: 0.144207954407
Inference time: 0.151439905167
Inference time: 0.144273996353
Inference time: 0.148524999619
Inference time: 0.148149967194
Inference ti

Inference time: 0.148198843002
Inference time: 0.146965026855
Inference time: 0.148215055466
Inference time: 0.143362998962
Inference time: 0.149543046951
Inference time: 0.142198085785
Inference time: 0.148449897766
Inference time: 0.149811983109
Inference time: 0.143348932266
Inference time: 0.149392843246
Inference time: 0.146301031113
Inference time: 0.147631168365
Inference time: 0.149004936218
Inference time: 0.144575119019
Inference time: 0.150028944016
Inference time: 0.142591953278
Inference time: 0.147588014603
Inference time: 0.146145105362
Inference time: 0.14543390274
Inference time: 0.147737979889
Inference time: 0.148772001266
Inference time: 0.14562702179
Inference time: 0.147044897079
Inference time: 0.145834922791
Inference time: 0.147239923477
Inference time: 0.148941040039
Inference time: 0.14315199852
Inference time: 0.148123979568
Inference time: 0.144712924957
Inference time: 0.148957014084
Inference time: 0.150144815445
Inference time: 0.14333987236
Inference ti

Inference time: 0.143580913544
Inference time: 0.147474050522
Inference time: 0.144629955292
Inference time: 0.148844003677
Inference time: 0.145747184753
Inference time: 0.145729064941
Inference time: 0.14940905571
Inference time: 0.143732070923
Inference time: 0.151233911514
Inference time: 0.147670984268
Inference time: 0.145083904266
Inference time: 0.147930145264
Inference time: 0.144994974136
Inference time: 0.15161895752
Inference time: 0.145017147064
Inference time: 0.149666070938
Inference time: 0.144867897034
Inference time: 0.147508859634
Inference time: 0.14982509613
Inference time: 0.143714189529
Inference time: 0.147754907608
Inference time: 0.146744012833
Inference time: 0.14736700058
Inference time: 0.148148059845
Inference time: 0.146943092346
Inference time: 0.142605066299
Inference time: 0.148911952972
Inference time: 0.143868923187
Inference time: 0.149194002151
Inference time: 0.146842002869
Inference time: 0.15003991127
Inference time: 0.149294137955
Inference tim

Inference time: 0.147989988327
Inference time: 0.145099878311
Inference time: 0.147673845291
Inference time: 0.14363193512
Inference time: 0.148452043533
Inference time: 0.143317937851
Inference time: 0.148345947266
Inference time: 0.146547079086
Inference time: 0.148239135742
Inference time: 0.147451877594
Inference time: 0.145813941956
Inference time: 0.147223949432
Inference time: 0.148963928223
Inference time: 0.148279905319
Inference time: 0.146454811096
Inference time: 0.14363193512
Inference time: 0.150591850281
Inference time: 0.146723985672
Inference time: 0.147186040878
Inference time: 0.146776914597
Inference time: 0.145549058914
Inference time: 0.147649049759
Inference time: 0.148250102997
Inference time: 0.144379854202
Inference time: 0.149035215378
Inference time: 0.146932840347
Inference time: 0.147392034531
Inference time: 0.147831916809
Inference time: 0.147311925888
Inference time: 0.14852809906
Inference time: 0.14372086525
Inference time: 0.150113105774
Inference ti

Inference time: 0.147674798965
Inference time: 0.150022983551
Inference time: 0.147607088089
Inference time: 0.149139165878
Inference time: 0.147766828537
Inference time: 0.144091129303
Inference time: 0.149647951126
Inference time: 0.14915394783
Inference time: 0.144198179245
Inference time: 0.150070905685
Inference time: 0.149609088898
Inference time: 0.149923086166
Inference time: 0.150544166565
Inference time: 0.146883010864
Inference time: 0.152575969696
Inference time: 0.14587688446
Inference time: 0.150271177292
Inference time: 0.145879983902
Inference time: 0.147516965866
Inference time: 0.150115966797
Inference time: 0.148561000824
Inference time: 0.150148868561
Inference time: 0.148420095444
Inference time: 0.147379875183
Inference time: 0.151229858398
Inference time: 0.146311044693
Inference time: 0.148497104645
Inference time: 0.149347066879
Inference time: 0.145592927933
Inference time: 0.150577068329
Inference time: 0.147704839706
Inference time: 0.148216962814
Inference 